# FeedForward 前馈神经网络
前馈神经网络（Feedforward Neural Network, FNN）是一种基础的神经网络结构，它的核心思想是通过线性和非线性变换将输入数据映射到输出。  

**（1）结构**  
- 输入层  
接受外部的输入信号，并将信号传递到网络的下一层。  

- 隐藏层  
在输入层和输出层之间，可以有多个隐藏层。每个隐藏层的节点将前一层的输出作为输入，进行一定的线性和非线性变换，然后将结果传递给下一层。  

- 输出层  
输出层的节点将最后一个隐藏层的输出作为输入，进行变换后，产生网络的输出。  

**（2）正向传播（Forward Propagation）**  
① 从输入层开始，将输入数据传递给第一个隐藏层。  
② 每个隐藏层的节点将前一层的输出作为输入，进行线性变换（例如，通过权重矩阵和偏置），然后应用一个非线性激活函数（例如ReLU或Sigmoid）。  
③ 重复步骤2，直到数据到达输出层。输出层通常会进行一次线性变换，然后可能会应用一个激活函数，例如softmax，以得到分类概率。  

**（3）反向传播（Backpropagation）**    
① 在网络的输出层，计算损失函数（例如，交叉熵损失）以衡量网络输出与真实标签的差异。  
② 从输出层开始，向后传播误差，计算每一层的梯度，并根据这些梯度更新网络参数。  

**（4）作用**  
- 函数逼近：  
FNN能够逼近任意复杂度的函数，是一种强大的函数逼近器。

- 分类和回归：  
可用于分类任务和回归任务。

- 特征转换：  
能够学习到输入数据的非线性特征表示。

In [ ]:
import torch.functional as F
import torch.nn as nn

In [ ]:
class FeedForward(nn.Module):
    def __init__(
        self,
        # 输入和输出的维度
        dim: int, 
        # 隐藏层的维度
        hidden_dim: int,
        # 用于调整隐藏层的维度，使其成为multiple_of的倍数。这可以确保在使用分布式处理或硬件优化时能够获得更好的性能。
        multiple_of: int, 
        # 用于控制隐藏层维度的缩放。如果提供了此参数，将使用此参数来乘以隐藏层的维度。
        ffn_dim_multiplier: Optional[float],
    ):
        super().__init__()
        # 将隐藏层维度设置为原始维度的2/3
        hidden_dim = int(2 * hidden_dim / 3)
        
        # 根据 ffn_dim_multiplier 进行缩放
        if ffn_dim_multiplier is not None:
            hidden_dim = int(ffn_dim_multiplier * hidden_dim)
            
        # 将hidden_dim调整为最接近的multiple_of的倍数（且不小于 hidden_dim）
        hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)
        '''知识补充：
        例如，假设我们的hidden_dim是820，而multiple_of是256。在这种情况下，我们期望hidden_dim是256的倍数。
        因此，我们需要找到一个最接近820的，且是256的倍数的数。那么根据这行代码，我们得到的新hidden_dim将会是1024（即256的四倍）。
        以下是这个计算过程的具体步骤：
        首先，我们计算hidden_dim + multiple_of - 1，这会得到820 + 256 - 1 = 1075。
        然后我们对这个结果进行整除操作//multiple_of，即1075//256=4（注意：//表示向下取整的除法）。
        最后我们把结果乘以multiple_of，得到的就是新的hidden_dim：4 * 256 = 1024。
        '''
        
        # 并行版本的全连接层，可以在多个设备上分别进行计算，以便处理大型模型。
        self.w1 = ColumnParallelLinear(
            dim, hidden_dim, bias=False, gather_output=False, init_method=lambda x: x
        )
        self.w2 = RowParallelLinear(
            hidden_dim, dim, bias=False, input_is_parallel=True, init_method=lambda x: x
        )
        self.w3 = ColumnParallelLinear(
            dim, hidden_dim, bias=False, gather_output=False, init_method=lambda x: x
        )
        
    
    def forward(self, x):
        '''前馈神经网络的过程，其中有两个隐藏层，并使用 SiLU 作为激活函数'''
        return self.w2(F.silu(self.w1(x)) * self.w3(x))
        '''知识补充: 
        F.silu(...): 对第一层的输出应用 Sigmoid Linear Unit (SiLU) 激活函数。
        SiLU 激活函数被定义为 f(x) = x * sigmoid(x)，其中 sigmoid(x) 是经典的 sigmoid 函数。
        这个函数的特点是能够将输入值映射到 0 到 1 的范围内。
        '''